# Summarizing cases using LLM
In this notebook, we conduct the summary of the cases using the Pegasus LLM, and examine the subsequent performance.

In [7]:
from transformers import pipeline, set_seed
import pandas as pd
from rouge_score import rouge_scorer



In [8]:
df = pd.read_csv("/Users/zeidsmac/Desktop/entropic_case/data/filtered_projects.csv") #change file path to local one

In [9]:
sample_text = df["description"][0]
sample_text

'Supporting the development and success of an industrial heat pump market through the develeopment of a standardized, modular, flexible compression heat pump technology. \n Incentive \n High temperature industrial heat pumps are an emerging technology that play a decisive role in providing a sustainable solution for industrial heat demand up to 200°C. Previous ISPT and TKI E&I projects (CATCH-IT, STEPS) have demonstrated the operation of pilot scale compression heat pumps for industrial applications, however CAPEX costs in the order of 500 €/kWth proved to be prohibitive for market introduction.  \n Market and cost reduction studies in the CRUISE project (ongoing) revealed that typical industrial capacities are 1-10 MWth and a lack of standardization for these units and the use of a flammable refrigerant were the most important cost determining factors. However, use of a non-flammable synthetic working medium (HFOs) still poses concerns about not being a long term sustainable solution.

Pegasus was choosen because in an earlier assignment at university, we examined the performance of it against different models (gpt2,t5,bart) and we found that it scored exceptionally better.

In [10]:
def summarize(text):
    pipe = pipeline("summarization", model="google/pegasus-cnn_dailymail")
    pipe_out = pipe(text)
    return pipe_out[0]["summary_text"].replace(" .<n>", ".\n")

In [11]:
df["summary"] = df["description_processed"].apply(summarize)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to 

In [27]:
df["summary"][0]

'High temperature industrial heat pumps are an emerging technology that play a decisive role in providing a sustainable solution for industrial heat demand up to 200C.\nPrevious ISPT and TKI E&I projects have demonstrated the operation of pilot scale compression heat pumps for industrial applications.\nTypical industrial capacities are 1-10 MWth and a lack of standardization for these units and the use of a flammable refrigerant were the most important cost determining factors .'

I made the summary on the preprocessed text, however it is not coherent, due to a lack of stopwords and punctuation. Therefore we regenerated the summaries based on the original text.

In [14]:
df["summary"] = df["description"].apply(summarize)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to 

In [28]:
df["summary"][0]

'High temperature industrial heat pumps are an emerging technology that play a decisive role in providing a sustainable solution for industrial heat demand up to 200C.\nPrevious ISPT and TKI E&I projects have demonstrated the operation of pilot scale compression heat pumps for industrial applications.\nTypical industrial capacities are 1-10 MWth and a lack of standardization for these units and the use of a flammable refrigerant were the most important cost determining factors .'

In [21]:
summary_len = len(df["summary"][0])
original_len = len(df["description"][0])
print(f"length of summary: {summary_len}. \nLength of original description: {original_len}.")

length of summary: 480. 
Length of original description: 3860.


# Generating summaries for the rest of the data
Since the data was ready, I thought I might as well create summaries for all 16 topics.

In [24]:
whole_df = pd.read_csv("/Users/zeidsmac/Desktop/entropic_case/data/ispt_processed.csv") #change file path to local one
whole_df["summary"] = whole_df["description"].apply(summarize)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to 

### Evaluating on ROUGE

As an extra step, I thought we might as well evaluate how summaries did based on the ROUGE metric. The scores are quite moderate, and can be greatly improved if the model were to be finetuned on some specific data for this case.

In [26]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Function to calculate ROUGE scores for pairs of text
def calculate_rouge_scores(reference, hypothesis):
    return scorer.score(reference, hypothesis)

# Applying the ROUGE evaluation for each row in the DataFrame
rouge_scores_list = df.apply(lambda row: calculate_rouge_scores(row['description'], row['summary']), axis=1)

# Creating a new DataFrame for the ROUGE scores
rouge_df = pd.DataFrame({
    'rouge1': rouge_scores_list.apply(lambda x: x['rouge1'].fmeasure),
    'rouge2': rouge_scores_list.apply(lambda x: x['rouge2'].fmeasure),
    'rougeL': rouge_scores_list.apply(lambda x: x['rougeL'].fmeasure)
})

# Display the new DataFrame with the ROUGE scores
rouge_df

,rouge1,rouge2,rougeL
0,0.218935,0.207715,0.218935
1,0.153846,0.145560,0.153846
2,0.395062,0.362500,0.395062
3,0.169492,0.160742,0.169492
